In [ ]:
#Convex MPC Stationkeeping for L2 Halo Orbit around L2 
# Halo Orbit generated through a differential corrector 

In [ ]:
#import the packages from the local environment
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))

In [ ]:
#packages used 
using LinearAlgebra
using ForwardDiff
using SatelliteDynamics
using DifferentialEquations
using Plots
using ECOS, Convex
using Mosek, MosekTools
using DelimitedFiles

In [ ]:
include("../src/dynamics.jl")
include("../src/parameters.jl")
include("../src/integrate.jl")
include("../src/solve.jl")

In [ ]:
Earth_Moon = ThreeBodySystem_EarthMoon()

In [ ]:
#use a richardson expansion as the initial guess for the differential corrector 
#saved as a .txt

#this reference trajectory was obtained through ThreeBodyProblem.jl
#halo from richardson expansion
richardson = readdlm("../refs/richardsonL2.txt", '\t', Float64, '\n')'

#period of the halo (in cr3bp units)
T = 3.4071472466192527

#period in days
T_scaled = T*Earth_Moon.time_scale

#initial condition in CR3BP units
x0_L2 = richardson[:,1]

#scaled initial condition
x0_L2_scaled = [x0_L2[1:3]*Earth_Moon.position_scale; x0_L2[4:6]*Earth_Moon.velocity_scale]

In [ ]:
Earth_Moon.acceleration_scale 

In [ ]:
#Integrate the dynamics with the initial state of the Richardson expansion as the first state
Φ_0 = vec(Matrix(1.0*I, 6,6))
x_0_stm = [x0_L2_scaled; Φ_0]

solution_in = dynamics_integrate(Earth_Moon ,x_0_stm, T_scaled)

all_state_in, stm_in, sol_t_in = get_state_stm(solution_in)

In [ ]:
stm_in 

In [ ]:
#this is not the same...
initial_norm = all_state_in[:,end] - all_state_in[:,1]

In [ ]:
#If we integrate the richardson expansion on the CR3bp dynamics, it is not completely periodic because it is only an approximation 
#this is a plot of the integrated orbit

plot(all_state_in[1,:], all_state_in[2,:], xlabel="x", ylabel="y")

In [ ]:
#the purpose of the differential corrector is to make the initial and final state of the orbit the same subject to some tolerance

#fix the richardson expansion guess with a single shooting differential corrector
function differential_corrector(system, x_0_stm, Tp)
    
    #this is the final time it converges to
    T_f = 0

    #Iterate to bring the δx at T/2 to the desired state
    max_iters = 1000

    #keep z0 fixed and change only x0 and y_dot_0
    for i=1:max_iters
    #for i=1:100

        solution = diff_solve(system, x_0_stm, Tp)

        #this is the state, STM, and time when y crosses the XZ plane 
        all_state, stm, sol_t = get_state_stm(solution)

        #display(stm)

        T2_state = all_state[:,end]

        #T2_state -> x,y,z, xdot, ydot, zdot
        #desired xdot an xzdot
        xz_d = [0, 0]

        rv = T2_state[1:6]

        rv_dot = three_body_prob_dynamics_scaled(system, rv)

        #desired state subtracted by state at t/2
        δxz = xz_d - [T2_state[4], T2_state[6]]

        #subset of the state transition matrix

        #all 6 variables 
        #stm_s = [stm[4,1] stm[4,3] stm[4,5]; stm[6,1] stm[6,3] stm[6,5]]

        #to change x and ydot 

        #keep z0 constant
        #stm_s = [stm[4,1] stm[4,5]; stm[6,1] stm[6,5]]
        #dy = [rv_dot[4]; rv_dot[6]]*[stm[2,1] stm[2,5]]

        #keep x0 constant
        stm_s = [stm[4,3, end] stm[4,5,end]; stm[6,3,end] stm[6,5,end]]
        dy = [rv_dot[4]; rv_dot[6]]*[stm[2,3,end] stm[2,5,end]]


        #println("this is stm s: ")
        #println(stm_s)


        #keep y0 constant
        #stm_s = [stm[4,1] stm[4,3]; stm[6,1] stm[6,3]]
        #dy = [rv_dot[4]; rv_dot[6]]*[stm[2,1] stm[2,3]]

        delta_x0 = (stm_s - dy/rv_dot[2])\δxz
        
        #println("THIS IS CONDITION NUMBER")
        #println(cond((stm_s - dy/rv_dot[2])))

        #delta_x0 = stm_s\δxz

        #update the initial state

        #x_0_stm[1] = x_0_stm[1] + delta_x0[1]
        x_0_stm[3] = x_0_stm[3] + delta_x0[1]
        x_0_stm[5] = x_0_stm[5] + delta_x0[2]

        #x_0_stm[1:6] = x_0_stm[1:6] + delta
        #when delta is below a set tolerance we are done

        println("residual: ", δxz)
        
        #loosen up the tolerance bc of the scaling of the problem
        if norm(δxz) < 1e-6

            #this is the total time, but we use it to ensure we have enough time to find where
            #y crosses the xz plane
            #print("this is sol t: ", sol_t)
            #updating time
                    
            T_f = sol_t[end]*2
            break
        end

    end
    
    corrected_solution = dynamics_integrate(system, x_0_stm, T_f)

    all_state_final, stm_final, sol_t_final = get_state_stm(corrected_solution)
    
    return all_state_final, stm_final, sol_t_final, corrected_solution
    
end

In [ ]:
#run the differential corrector to get the corrected solution
all_state_final, stm_final, sol_t_final, full_solution = differential_corrector(Earth_Moon, x_0_stm, T_scaled)

In [ ]:
all_state_final 

In [ ]:
#this is the halo after the differential corrector 
plot(all_state_final[1,:], all_state_final[2,:], all_state_final[3,:])

In [ ]:
all_state_final[:,1] 

In [ ]:
#i don't think we use this trajectory, will prob ciomment out...

#create a 2 orbit horizon reference trajectory
# periodic_initial_condition2 = all_state_final[:,1]

# #Integrate the dynamics with the perturbed initial condtition
# Φ_0 = vec(Matrix(1.0*I, 6,6))
# x_0_periodic2 = [periodic_initial_condition2; Φ_0]

# #integrate for 2 revs
# solution_periodic2 = dynamics_integrate(Earth_Moon, x_0_periodic2, 2*sol_t_final[end])

# all_state_2, stm_in_2, sol_t_in_2 = get_state_stm(solution_periodic2)

In [ ]:
#Check when the orbit leaves
periodic_initial_condition = all_state_final[:,1]

#Integrate the dynamics with the perturbed initial condtition
Φ_0 = vec(Matrix(1.0*I, 6,6))
x_0_periodic = [periodic_initial_condition; Φ_0]

#integrate for 3.5 revs
solution_periodic = dynamics_integrate(Earth_Moon, x_0_stm, 3.5*sol_t_final[end])

all_state_lp, stm_in_lp, sol_t_in_lp = get_state_stm(solution_periodic)

In [ ]:
#the orbit after 3.5 revs 
plot(all_state_lp[1,:], all_state_lp[2,:], all_state_lp[3,:], aspectratio=true)

In [ ]:
#as seen above, after about 3.5 periods it begins to diverge from the periodic orbit 
#will formulate as a convex optimiztion problem to generate the station keeping 
#maneuvers

In [ ]:
nx = 6 #number of states
nu = 3 #number of controls 

N_period = 41  #41 knot points per period per period

#the first state repeats with the last state
N_h = 81 #horizon 2 periods

#timestep 
Δt = (sol_t_final[end]/(N_period-1))

In [ ]:
#variables to save all the dynamics jacobians this is along 1 period of the reference orbit
all_Ad = zeros(6,6,N_period)
all_Bd = zeros(6,3,N_period)

In [ ]:
#this is the corrected period from the differential corrector
T_corrected = sol_t_final[end]

In [ ]:
#create even timesteps along the horizon 
horizon = LinRange(0, T_corrected, N_period)

In [ ]:
#used to save the state transition matrix at each knot point
stm_times = zeros((6,6,N_period))

In [ ]:
#get the reference trajectory and state transition matrices at the correct times along the horizon 
reference_trajectory = zeros(nx, N_period)

for k=1:N_period

    reference_trajectory[:,k] = full_solution(horizon).u[k][1:6]
    
    #save the reference state transition matrices at the dedicated timesteps
    
    stm_times[:,:,k] = reshape(full_solution(horizon).u[k][7:end], (6,6)) 
        
end

In [ ]:
#monodromy is the stm at the end of the periodic orbit
monodromy_eigen = eigen(stm_times[:,:,end])

stable_dir = real(monodromy_eigen.vectors[:,1])
unstable_dir = real(monodromy_eigen.vectors[:,end])
#find the eigen values of the monodromy

In [ ]:
#save the unstable directions in a matrix
unstable_directions = zeros(6,N_period)

In [ ]:
#the unstable directions are then the stm at the timestep multiplied by the unstable direction found by the monodromy
for k = 1:N_period
    
    unstable_directions[:,k] = (stm_times[:,:,k]*unstable_dir)/norm(stm_times[:,:,k]*unstable_dir)
    
end

In [ ]:
#unstable directions for the unstable manifolds along the orbit
unstable_directions

In [ ]:
#save the manifolds when perturbation is added 
all_manifolds_plus = Matrix{Float64}[]

#save the manifolds when perturbation is subtracted 
all_manifolds_minus = Matrix{Float64}[]

In [ ]:
#plot the manifolds
#+ manifold to the right
# - manifold to the left

#epsilon multiplied by the perturbation when finding the manifolds 
epz = 3

plot(reference_trajectory[1,:], reference_trajectory[2,:], label=nothing)

all_state_mplus = 0

all_state_mminus = 0
for i=1:N_period
 
    initial_state_plus = reference_trajectory[:,i] + epz.*[unstable_directions[1:3,i];zeros(3)]

    initial_state_minus = reference_trajectory[:,i] - epz.*[unstable_directions[1:3,i];zeros(3)]

    inv_manifold_plus = just_dynamics_integrate(Earth_Moon, initial_state_plus, 1.5*sol_t_final[end])

    inv_manifold_minus = just_dynamics_integrate(Earth_Moon, initial_state_minus, 1.45*sol_t_final[end])

    all_state_mplus = get_state(inv_manifold_plus)

    all_state_mminus = get_state(inv_manifold_minus)
    

    push!(all_manifolds_plus, all_state_mplus)
    push!(all_manifolds_minus, all_state_mminus)


    plot!(all_state_mplus[1,:], all_state_mplus[2,:], color="red", label=nothing)
    plot!(all_state_mminus[1,:], all_state_mminus[2,:], color="orange", label=nothing)

end


plot!(all_state_mplus[1,:], all_state_mplus[2,:], color="red", label = "Unstable Invariant Manifold +")
plot!(all_state_mminus[1,:], all_state_mminus[2,:], color="orange", label = "Unstable Invariant Manifold -")

plot!(reference_trajectory[1,:], reference_trajectory[2,:], label="Reference Halo Orbit", linewidth = 5, xlabel = "X (km)", ylabel = "Y (km)", color="blue")
orbit_manifolds = scatter!([Earth_Moon.XL2[1]*Earth_Moon.position_scale], [0], markercolor = "green", label = "L2", legend=true, title = "L2 Halo Orbit with Unstable Invariant Manifolds")

In [ ]:
#get the discrete dynamics jacobians for the entire reference trajectory along the knot points
for i=1:N_period

    all_Ad[:,:,i] = ForwardDiff.jacobian(x_-> RK4_integrator(Earth_Moon, x_, zeros(3)), reference_trajectory[:,i])


    all_Bd[:,:,i] = ForwardDiff.jacobian(u_-> RK4_integrator(Earth_Moon, reference_trajectory[:,i], u_), zeros(3))
    
    
end

In [ ]:
#our horizon is 2 revolutions, therefore we need jacobians along the entire horizon
#will concatenate jacobians for two periods

all_Ad_1 = cat(all_Ad, all_Ad[:,:,2:end], dims=3)
all_Bd_1 = cat(all_Bd, all_Bd[:,:,2:end], dims=3)

In [ ]:
#compute backward ricatti recursion

#cost to go matrices for 16 revs
P = zeros((6,6,N_period+((N_period-1)*15))) #size state x state (6x6) 
K = zeros((3,6,N_period+((N_period-1)*15))) #size control x state (3x6)

#state cost weight matrix
Q = 1.0*Matrix(I,6,6).*1e-3

#terminal cost weight matrix
Qf = 1.0*Matrix(I,6,6).*1e-3

#control cost weight matrix
R = 1.0*Matrix(I,3,3).*1e3

In [ ]:
#variable to concatenate 16 sets of jacobians for 16 revolutions. this is to compute the recursion until we get a periodic pattern
all_Ad_16 = all_Ad
all_Bd_16 = all_Bd

In [ ]:
for i = 2:16
    all_Ad_16 = cat(all_Ad_16, all_Ad[:,:,2:end], dims = 3)
    all_Bd_16 = cat(all_Bd_16, all_Bd[:,:,2:end], dims = 3)
end

In [ ]:
#Ricatti recursion over approximately 7 periods
P[:,:,end] = Qf

for k = reverse(1:640)
    #println(k)    
    K[:,:,k] = (R + all_Bd_16[:,:,k]'*P[:,:,k+1]*all_Bd_16[:,:,k])\(all_Bd_16[:,:,k]'*P[:,:,k+1]*all_Ad_16[:,:, k]) # ricatti recursion for both A and B matrices changing
    P[:,:,k] = Q + all_Ad_16[:,:,k]'*P[:,:,k+1]*all_Ad_16[:,:,k] - all_Ad_16[:,:,k]'*P[:,:,k+1]*all_Bd_16[:,:,k]*K[:,:,k]
        
end

In [ ]:
P[:,:,1]

In [ ]:
#can plot each diagonal element of the cost to go to see that it is periodic..
plot(P[1,1,:])
#plot(P[2,2,:])
#plot(P[3,3,:])
#plot(P[4,4,:])
#plot(P[5,5,:])
#plot(P[6,6,:])

In [ ]:
#cost to go converges and is periodic! will get one orbit of the cost to go to make a constraint
V = P[:,:,1:41]

In [ ]:
#cost to go matrices for 2 orbits worth of data
V_2orbits = cat(V, V[:,:,2:end], dims=3)

#unstable manifold directions for 2 orbits worth of data 
unstable_directions_2 = cat(unstable_directions, unstable_directions[:, 2:end], dims=2)

In [ ]:
#define the injection error, how far we are off from the initial condition of the halo 

#in cr3bp units 
x_initial_ref = [1e-6, 0, 0, 0, 1811.22e-6, 0]

#in km and days 
x_initial_ref_scaled = [x_initial_ref[1:3]*Earth_Moon.position_scale; x_initial_ref[4:6]*Earth_Moon.velocity_scale]

In [ ]:
#define the tube radii

#position radius
tube_pose_r = 1000

#velocity radius 
tube_vel_r = 1000

In [ ]:
#find the axis lengths for each of these P matrices

all_axes_length = zeros(6, 81) 

for i= 1:81

    P_i = V_2orbits[:,:,i]

    vals_i = eigen(P_i).values  

    axes_length_i = 2 ./ (sqrt.(vals_i))

    all_axes_length[:,i] = axes_length_i    

end

#velocity axes lenghts are much smaller than position since a larger deviation will lead to a larger cost in the long run

In [ ]:
#this is the reference trajectory for 2 orbits starting at the beginning
reference_trajectory2 = [reference_trajectory reference_trajectory[:,2:end]]


reference_trajectory_half = cat(reference_trajectory[:,21:end], reference_trajectory[:,2:21], dims=2)

#this is the reference trajectory starting at the half, timesteo 21
reference_trajectory2_2 = cat(reference_trajectory_half, reference_trajectory_half[:,2:end], dims=2)

In [ ]:
#this scales the axes on the cost-to-go ellipsoid. 
epsilon = 1e4

#parameter for the contingency constraint 
a = 1e-2

In [ ]:
#define the state constraint type

#"euc" for the Euclidean ball constraint 
constraint_type = "euc"

#"ctg" for the cost-to-go ellipsoid constraint 
#constraint_type = "ctg"

In [ ]:
#one solve test
display("updating prob")
cons, X_1, U_1 = update_prob(Earth_Moon, x_initial_ref_scaled, all_Ad_1, all_Bd_1, V_2orbits, unstable_directions_2, reference_trajectory2, constraint_type)
display("solving")
display(X_1)
Xm_1, Um_1 = solve_opt(cons, X_1, U_1, N_h)

In [ ]:
Xm_1 

In [ ]:
Um_1 

In [ ]:
#check the constraint satisfaction on the manifold constraint
# constraint_satisfied = zeros(N_h)

# for i=1:N_h
#     constraint_satisfied[i] = dot(Xm[:,i], unstable_directions_2[:,i])
# end

In [ ]:
#all_Ad_1 and all_Bd_1 are the set of jacobians for two revoltions when we start at the beginning of the orbit 
#all_Ad_2 and all_Bd_2 are the set of jacobians for two revolutions when we start in the middle of the orbit 

all_Ad_half = cat(all_Ad[:,:,21:end], all_Ad[:,:,2:21], dims=3)
all_Bd_half = cat(all_Bd[:,:,21:end], all_Bd[:,:,2:21], dims=3)

all_Ad_2 = cat(all_Ad_half, all_Ad_half[:,:,2:end], dims = 3)
all_Bd_2 = cat(all_Bd_half, all_Bd_half[:,:,2:end], dims=3)

#also need to get the unstable directions and cost-to-go at the half point for two revoltions

#unstable directions is good. Need to fix V_2orbits
unstable_directions_half = cat(unstable_directions[:,21:end], unstable_directions[:,2:21], dims=2)

#P is for the 16 orbits. Need the one for just 1 orbit
V_half = cat(V[:,:,21:end], V[:,:,2:21], dims=3)


#unstable directions if you start from the beginning
unstable_directions_2_1 = unstable_directions_2
#this is the unstable direction if you start at the half
unstable_directions_2_2 = cat(unstable_directions_half, unstable_directions_half[:,2:end], dims=2)


#cost to go if you start from the beginning
V_2orbits_1 = V_2orbits

#cost to go in you start at the half
V_2orbits_2 = cat(V_half, V_half[:,:,2:end], dims=3)

In [ ]:
#run the MPC 

orbit_count = 0

#number of orbits you want to plan go for
orbit_num = 100

halforbit_num = orbit_num*2 

#this is for each orbit plan

optimal_Δx = zeros(nx, N_period)
optimal_u = zeros(nu, N_period-1)

#data for all the orbits
all_optimal_Δx = zeros(nx, 21, halforbit_num)
all_optimal_x = zeros(nx, 21, halforbit_num)
all_optimal_u = zeros(nu, 20, halforbit_num)

#save the solution of each solve 
solution_xm = zeros(6, 81, halforbit_num)
solution_um = zeros(6, 80, halforbit_num)

 
for i=1:halforbit_num

    
    #if you start from the beginning use the appropriate set of jacobians
    if i%2 != 0
        #println("using jacobian set 1")
        all_Ad_k = all_Ad_1
        all_Bd_k = all_Bd_1
        unstable_directions_k = unstable_directions_2_1
        P_k = V_2orbits_1 
        reference_traj_k = reference_trajectory2       
    else
        #println("using jacobian set 2")
        all_Ad_k = all_Ad_2
        all_Bd_k = all_Bd_2
        unstable_directions_k = unstable_directions_2_2
        P_k = V_2orbits_2
        reference_traj_k = reference_trajectory2_2
    end
    
    #the zeros is the bias 
    cons, X, U = update_prob(Earth_Moon, x_initial_ref_scaled, all_Ad_k, all_Bd_k, P_k, unstable_directions_k, reference_traj_k, constraint_type)
        
    Xm, Um = solve_opt(cons, X, U,N_h)

    solution_xm = Xm
    solution_um = Um
    
    #get the optimal controls for half a period (20 for this example)
    optimal_u = Um[:,1:20]
        
    #size N_period
        
    orbit_count += 1
    
    #rollout on the nonlinear dynamics
    if i%2 != 0
        
        #print("from beginning")
        xtraj_halfperiod = integrate_halfperiod(Earth_Moon, reference_trajectory[:,1] + Xm[:,1], optimal_u)

        #this is the deviation from the reference
        all_optimal_Δx[:,:,orbit_count] = xtraj_halfperiod - reference_trajectory[:,1:21]
        
        #integrate the initial condition by all the optimal controls from the solver        
    else
        
        #println("from half")

        #this Xm is the one that just got computed
        xtraj_halfperiod = integrate_halfperiod(Earth_Moon, reference_trajectory[:,21] + Xm[:,1], optimal_u)

        all_optimal_Δx[:,:,orbit_count] = xtraj_halfperiod - reference_trajectory[:,21:41] 
        #integrate the initial condition by all the optimal controls from the solver
        
    end
    
    all_optimal_x[:,:,orbit_count] = xtraj_halfperiod
    
    all_optimal_u[:,:,orbit_count] = optimal_u
    
    println("Half Revolution solved: ", i)
    
    #update the intiial condition for the next rev (has to be a delta)
    x_initial_ref_scaled = all_optimal_Δx[:,end, orbit_count]

    
end  

In [ ]:
all_optimal_u

In [ ]:
#format the outputin 3D arrays. put all the half trajectory solutions together 
all_optimal_x_formatted = zeros(6, N_period, orbit_num)
all_optimal_Δx_formatted = zeros(6, N_period, orbit_num)

In [ ]:
for i=1:orbit_num
    
    all_optimal_x_formatted[:,1:21,i] = all_optimal_x[:,:,(2*i-1)]
    
    #2:end is used to not repeat the same element twice
    all_optimal_x_formatted[:,22:end,i] = all_optimal_x[:,2:end,(2*i)]
    
    
    all_optimal_Δx_formatted[:,1:21,i] = all_optimal_Δx[:,:,(2*i-1)]
    
    all_optimal_Δx_formatted[:,22:end,i] = all_optimal_Δx[:,2:end,(2*i)]
    
end

In [ ]:
#format to 2D arrays 
all_xtraj = reshape(all_optimal_x_formatted, (6, N_period*orbit_num))

all_Δxtraj = reshape(all_optimal_Δx_formatted, (6, N_period*orbit_num))

all_utraj = reshape(all_optimal_u, (3, (N_period-1)*orbit_num))

In [ ]:
normz = zeros(4100)
for i=1:4100
    normz[i] = norm(all_Δxtraj[:,i])
end

In [ ]:
#state norm for the entire trajectory 
plot(normz )

In [ ]:
#horizon for a 10 revolution trajectory in days
days_range = range(0,409, 410)

days = collect(days_range)*Δt

In [ ]:
Δt 

In [ ]:
#this plots orbits 10-20

#for euclidean constraint 
# plot_range_x = (-1.5e-8, 1e-8)
# plot_range_y = (-1.5e-8, 1e-8)
# plot_range_z = (-1.5e-8, 1e-8)


#for ctg constraint 
plot_range_x = (-1e-9, 1.5e-9)
plot_range_y = (-1e-8, 1e-8)
plot_range_z = (-1.5e-8, 1e-8)


plot1 = plot(days, all_utraj[1,410:819]*1e3/86400^2, linewidth = 2,ylim = plot_range_x, title= "X Control")
plot2 = plot(days, all_utraj[2,410:819]*1e3/86400^2, linewidth = 2, ylim = plot_range_y, title= "Y Control", linecolor="red")
plot3 = plot(days , all_utraj[3,410:819]*1e3/86400^2, linewidth=2, ylim = plot_range_z, title= "Z Control", linecolor="green")


plot_layout = @layout [a;
                       b; 
                       c]

control1020 = plot(plot1, plot2, plot3, layout=plot_layout, legend=false, xlabel="Days", xlabelfontsize=8, ylabel="Force (N)", ylabelfontsize=8, ytickfontsize = 6)

In [ ]:
plot(all_utraj[1,410:430]) 

In [ ]:
plot(all_utraj[1, 410:450])

In [ ]:
plot(all_utraj[2, 410:450])

In [ ]:
all_utraj[2, 410:429] 

In [ ]:
all_utraj[1, 410:429]

In [ ]:
#burn locations plot
burn_locations = plot(reference_trajectory[1,:], reference_trajectory[2,:], reference_trajectory[3,:], linewidth = 3, title= "Burn Locations", label = "L2 Halo Orbit", legend=:topright)
scatter!([reference_trajectory[1,12]],[reference_trajectory[2,12]],[reference_trajectory[3,12]], label="X Burn", markersize = 7, xlabel="X [km]", ylabel = "                 Y [km]", zlabel="Z [km]", 
ylabelfontsize=6, ytickfontsize = 6, xlabelfontsize=6, xtickfontsize = 6, zlabelfontsize=6, ztickfontsize = 6)
scatter!([reference_trajectory[1,12], reference_trajectory[1,14], reference_trajectory[1,31]],[reference_trajectory[2,12], reference_trajectory[2,14], reference_trajectory[2,31]],[reference_trajectory[3,12],reference_trajectory[3,14], reference_trajectory[3,31]], label="Y Burn", markersize = 4)

In [ ]:
#plot the 100 orbits

plot(all_xtraj[1,:], all_xtraj[2,:], all_xtraj[3,:], label="MPC Trajectory", title="MPC Trajectory")

scatter!([all_xtraj[1,1]], [all_xtraj[2,1]], [all_xtraj[3,1]], label="Start")

plot!(reference_trajectory[1,:], reference_trajectory[2,:], reference_trajectory[3,:], label = "Reference Trajectory")

In [ ]:
#this is already in units of km/(days)^2
scaled_control = all_utraj

In [ ]:
#integrate over the timestep to get the velocity cost in km/day
x_cost = sum(abs.(scaled_control[1,:]))*(Δt)
y_cost = sum(abs.(scaled_control[2,:]))*(Δt)
z_cost = sum(abs.(scaled_control[3,:]))*(Δt)

In [ ]:
#velocity cost in m/s
#dividing by 86400 to get into seconds bc currently it is in days

#units are good (checked in simulation_test)
x_cost_ms = x_cost*(1e3/86400)
y_cost_ms = y_cost*(1e3/86400)
z_cost_ms = z_cost*(1e3/86400)

display(x_cost_ms)
display(y_cost_ms)
display(z_cost_ms)

In [ ]:
all_cost = x_cost_ms + y_cost_ms + z_cost_ms  

In [ ]:
cost_per_rev = all_cost/orbit_num

In [ ]:
#Exclude the first 10 timesteps to fix the injection error
x_cost_good = sum(abs.(scaled_control[1,42:end]))*(Δt) *(1e3/86400)
y_cost_good = sum(abs.(scaled_control[2,42:end]))*(Δt) *(1e3/86400)
z_cost_good = sum(abs.(scaled_control[3,42:end]))*(Δt) *(1e3/86400)

display(x_cost_good)
display(y_cost_good)
display(z_cost_good)

In [ ]:
# #cost for 95 revs
all_cost_good = x_cost_good + y_cost_good + z_cost_good 

In [ ]:
#get the cost per year 

year_constant = 365/sol_t_final[end] 

year_cost = cost_per_rev*year_constant 

In [ ]:
#check the constraint for the manifold direction 
constraint_check = zeros(4100)
for k=1:100
    for i=1:41

        constraint_check[41*(k-1)+i] = dot(unstable_directions[:,i], all_Δxtraj[:, 41*(k-1)+i]) #> 0 

    end
end

In [ ]:
#first timestep not included in the constraint 
plot(constraint_check[2:end].>0)

In [ ]:
all_utraj 

In [ ]:
all_utraj[:,1]  

In [ ]:
all_utraj[:,2]  

In [ ]:
all_utraj[:,3]  